# Current Data Sources

1. Google Trends API
2. Twitter API

# Current Features
1. term	
2. current_popularity	
3. change_3mo	
4. change_6mo	
5. change_9mo	
6. change_12mo	
7. change_24mo	
8. popularity_2y	
9. sentiment
10. subjectivity

# Visulizations to make...
1. Sentiment PDF
2. popularity and predicted popularity

# Features to add...
1. sentiment standard deviation

In [92]:
#imports and instantiations
from pytrends.request import TrendReq
import tweepy
from statsmodels.tsa import ar_model, stattools, arima_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from textblob import TextBlob
import pandas as pd
from potosnail import Stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

In [2]:
#helper functions
def GetReport(keywords, span='today 5-y', geo='', quiet=True):
    '''observe a search term's popularity in the past 5 years'''
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(keywords, cat=0, timeframe=span, geo=geo, gprop='')
    ts = pytrends.interest_over_time().drop(['isPartial'], axis='columns')
    if quiet == False:
        print(ts.plot())
    return ts

def AnalyzeTwitter(keyword):
    '''find the average sentimental value and subjectivity of a given search term'''
    c1 = 'aHXduTrDkva3ItY52tUtYVPvA'
    c2 = 'Qs6d4oNT3zXxDqOhita7IG07CfAJGceoqIs1sGuA4OURlbLP6d'
    a1 = '1181578611171762177-sGQaj7E9fpWi2aEB3MfWL4nTRovXYk'
    a2 = 'wa77yBJZJSOKOAzdaJYDruc9U1HrGhzyDhWgKvSQpm2hv'
    auth = tweepy.OAuthHandler(c1, c2)
    auth.set_access_token(a1, a2)
    api = tweepy.API(auth)
    topic = api.search(keyword)
    sent = 0
    sub = 0
    sents = []
    for i in range(len(topic)):
        tweet = topic[i]._json['text'].replace('@', '')
        blob = TextBlob(tweet)
        sents.append(blob.sentiment[0])
        sent += blob.sentiment[0]/len(topic)
        sub += blob.sentiment[1]/len(topic)
    return sent, sub, sents

def Collect(keyword, quiet=True):
    '''tells us how popularity for a given search term is expected to change'''
    row = {}
    ts = GetReport([keyword])
    row['term'] = keyword
    model = ar_model.AutoReg(ts, lags=4).fit()
    pred = model.predict(start=260, end=356)
    current_popularity = np.array(ts)[-1][0]
    row['current_popularity'] =  current_popularity
    row['change_3mo'] = '{}%'.format(round(((pred[11] - current_popularity) / current_popularity) * 100, 1))
    row['change_6mo'] = '{}%'.format(round(((pred[23] - current_popularity) / current_popularity) * 100, 1))
    row['change_9mo'] = '{}%'.format(round(((pred[35] - current_popularity) / current_popularity) * 100, 1))
    row['change_12mo'] = '{}%'.format(round(((pred[47] - current_popularity) / current_popularity) * 100, 1))
    row['change_24mo'] = '{}%'.format(round(((pred[95] - current_popularity) / current_popularity) * 100, 1))
    row['popularity_2y'] = round((((pred[95] - current_popularity) / current_popularity) + 1) * current_popularity)
    estimated_reach = None
    estimated_clicks = None
    sentiment, subjectivity, sentiments = AnalyzeTwitter(keyword)
    row['sentiment'] = round(sentiment, 2)
    row['subjectivity'] = round(subjectivity, 2)
    row['sentiments_std'] = round(np.std(sentiments), 2)
    if quiet == True:
        return row
    else:
        ts['date'] = ts.index 
        pred2 = pd.DataFrame(pred)
        pred2.columns = [keyword]
        pred2['date'] = pred.index
        total = pd.concat([ts, pred2])
        return total, row

def CollectLoop(terms_list):
    '''tells us how popularity for a given list of search terms are expected to change'''
    df = pd.DataFrame(Collect(terms_list[0]), index=[0])
    for term in terms_list[1:]:
        temp = pd.DataFrame(Collect(term), index=[0])
        df = pd.concat([df, temp])
    return df.reset_index().drop(['index'], axis='columns')

def PlotOne(keyword):
    '''the output a user gets when looking at one term'''
    ts, results = Collect(keyword, quiet=False)
    subj = results['subjectivity']
    obj = 1 - subj
    X = ['%subjective', '%objective']
    y = [subj, obj]
    X2 = ['sentiment']
    y2 = results['sentiment']
    if results['popularity_2y'] > results['current_popularity']:
        future = 'increase'
    else:
        future = 'decrease'
    fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = results['sentiment'],
    mode = "gauge+number",
    title = {'text': "Sentiment of '{}' based on tweets".format(keyword)},
    gauge = {'axis': {'range': [-1, 1]},
             'steps' : [
                 {'range': [-1, 0], 'color': "red"},
                 {'range': [0, 1], 'color': "lightgreen"}]}))
    fig.show()
    fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = results['subjectivity'],
    mode = "gauge+number",
    title = {'text': "Subjectivity of '{}' based on tweets".format(keyword)},
    gauge = {'axis': {'range': [0, 1]},
             'steps' : [
                 {'range': [0, 0.5], 'color': "yellow"},
                 {'range': [0.5, 1], 'color': "blue"}]}))
    fig.show()
    fig = px.line(ts, x='date', y=keyword, range_y=[0, 100])
    fig.show()
    
def PlotMany(keywords):
    '''the output a user gets when looking at multiple terms'''
    ts = GetReport(keywords)
    results = CollectLoop(keywords)
    fig = px.bar(results, x='term', y='popularity_2y', 
                 color='current_popularity', title='Predicted Search Popularity in the next 2 years', range_y=[0, 100])
    fig.show()
    sentiments = []
    stds = []
    for keyword in keywords:
        sentiment , _, sents = AnalyzeTwitter(keyword)
        sentiments.append(sentiment)
        stds.append(np.std(sents))
    twitter = {'term': products, 'sentiment': sentiments, 'std': stds}
    df = pd.DataFrame(twitter)
    fig = px.bar(df, x='term', y='sentiment', 
                     color='std', title='Twitter sentiment', range_y=[-1, 1])
    fig.show()
    results['new'] = results['change_24mo'].apply(lambda x: float(x[:-1]))
    best = results.loc[results['new']==max(results['new'])]['term'].iloc[0]
    print(PlotOne(best))

In [3]:
hobbies = ['biking', 'gardening', 'surfing', 'home design', 'workouts']
products = ['face mask', 'bike helmet', 'board shorts', 'back brace', 'puzzles']

## How can this output be improved?
* improve the model and take seasonality into account
* make column names more user friendly

# Modeling
* Step 1: select 4 search term examples; an upward trend, a seasonal, a '2020' spike or dip, and a no trend
* Step 2: inspect acf and pacf for each
* Step 3: check for seasonality and stationarity
* Step 4: select the best model for each
* Step 5: Automate this process with OOP

## selecting search term examples

In [31]:
upward = GetReport(['mom jeans'])
seasonal = GetReport(['sleeping bags'])
spike_2020 = GetReport(['face mask'])
little_trend = GetReport(['ashtray'])

# Checking acf and pacf

In [33]:
upward_acf = stattools.acf(upward)
seasonal_acf = stattools.acf(seasonal)
spike_2020_acf = stattools.acf(spike_2020)
little_trend_acf = stattools.acf(little_trend)

upward_pacf = stattools.pacf(upward)
seasonal_pacf = stattools.pacf(seasonal)
spike_2020_pacf = stattools.pacf(spike_2020)
little_trend_pacf = stattools.pacf(little_trend)

# Modeling process
## for a given search term

1. check for seasonality
2. use ARIMA if there is no seasonality, otherwise use SARIMA
3. optimize lags programatically, use acf and pacf if nessecary
4. evaluate with AIC and BIC

helpful reference: https://towardsdatascience.com/time-series-essentials-fe6727ab6a94